In [0]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold,StratifiedKFold, cross_val_score, train_test_split


import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier,Pool, cv

from sklearn.metrics import mean_squared_error, accuracy_score, roc_auc_score, f1_score

In [1]:
!pip install bayesian-optimization

  Running setup.py bdist_wheel for bayesian-optimization: started
  Running setup.py bdist_wheel for bayesian-optimization: finished with status 'done'
  Stored in directory: C:\Users\kumarabh\AppData\Local\pip\Cache\wheels\1d\0d\3b\6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
from bayes_opt import BayesianOptimization

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
import os
print(os.listdir("../content/gdrive/My Drive/Colab Notebooks/AmExpert"))

['testF.csv', 'trainF.csv', 'test.csv', 'Norm_LGBC.csv', 'AmExpert_Redemption.ipynb', 'testX.csv', 'trainX.csv']


In [0]:
train = pd.read_csv("../content/gdrive/My Drive/Colab Notebooks/AmExpert/trainF.csv")
test = pd.read_csv("../content/gdrive/My Drive/Colab Notebooks/AmExpert/testF.csv")

In [0]:
test1 = pd.read_csv("../content/gdrive/My Drive/Colab Notebooks/AmExpert/test.csv")

In [0]:
train.shape, test.shape

((78369, 66), (50226, 65))

In [0]:
train['redemption_status'].value_counts()

0    77640
1      729
Name: redemption_status, dtype: int64

In [0]:
target = train['redemption_status']
train.drop('redemption_status', axis = 1, inplace = True)

In [0]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
train[train.columns]=sc.fit_transform(train)
test[test.columns]=sc.transform(test)

In [0]:
import xgboost as xgb
import warnings

In [0]:
dtrain = xgb.DMatrix(train, label=target)
#dtest = xgb.DMatrix(x_test)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [0]:
AUCbest = -1
ITERbest = 0

In [0]:
dtrain = xgb.DMatrix(train, label=target)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [0]:
def xgb_evaluate(n_estimators,num_leaves,max_depth, gamma, subsample, colsample_bytree, min_child_weight, max_delta_step,learning_rate, reg_alpha, reg_lambda):
    global AUCbest
    global ITERbest
    params = {
              'booster' : 'gbtree',
              'n_estimators': int(n_estimators),
              'num_leaves' : int(num_leaves),
              'max_depth' : int(max_depth),
              'gamma' : gamma,
              'eta' : 0.1,
              'objective' : 'binary:logistic',
              'nthread' : 4,
              'silent' : True,
              'eval_metric': 'auc',
              'subsample' : max(min(subsample, 1), 0),
              'colsample_bytree' : max(min(colsample_bytree, 1), 0),
              'min_child_weight' : min_child_weight,
              'max_delta_step' : int(max_delta_step),
              'learning_rate' : learning_rate,
              'reg_alpha': reg_alpha,
              'reg_lambda': reg_lambda,
              'scale_pos_weight': 106,
              'seed' : 1993
              }
    
    xgbc = xgb.cv(params, dtrain, num_boost_round=1000,early_stopping_rounds = 100,stratified = True, nfold=3)    
    
    val_score = xgbc['test-auc-mean'].iloc[-1]
    train_score = xgbc['train-auc-mean'].iloc[-1]
    
    print(' Stopped after %d iterations with train-auc = %f val-auc = %f ( diff = %f ) train-gini = %f val-gini = %f' % ( len(xgbc), train_score, val_score, (train_score - val_score), (train_score*2-1),
    (val_score*2-1)) )
    if ( val_score > AUCbest ):
        AUCbest = val_score
        ITERbest = len(xgbc)

    return (val_score*2) - 1

In [0]:


xgb_bo = BayesianOptimization(xgb_evaluate, {
                                            'n_estimators':(1000,3000),
                                            'num_leaves' : (20, 100),
                                            'max_depth': (1, 20),
                                            'gamma': (0.001, 10.0),
                                            'min_child_weight': (0, 20),
                                            'max_delta_step': (0, 10),
                                            'subsample': (0.4, 1.0),
                                            'colsample_bytree' :(0.4, 1.0),
                                            'learning_rate' : (0.01, 0.5),
                                            'reg_alpha': (0.0, 10),
                                            'reg_lambda': (0.0,10)
                                             
                                            }, random_state = 1993)
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
with warnings.catch_warnings():
     warnings.filterwarnings('ignore')
     xgb_bo.maximize(init_points=3, n_iter=25, acq='ei')

|   iter    |  target   | colsam... |   gamma   | learni... | max_de... | max_depth | min_ch... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
 Stopped after 4 iterations with train-auc = 0.949949 val-auc = 0.932545 ( diff = 0.017404 ) train-gini = 0.899897 val-gini = 0.865090
|  1        |  0.8651   |  0.4603   |  2.195    |  0.4662   |  3.314    |  4.886    |  3.368    |  2.014e+0 |  78.06    |  5.708    |  1.846    |  0.9754   |
 Stopped after 9 iterations with train-auc = 0.947207 val-auc = 0.934662 ( diff = 0.012545 ) train-gini = 0.894413 val-gini = 0.869323
|  2        |  0.8693   |  0.7355   |  8.388    |  0.292    |  5.537    |  3.048    |  6.129    |  1.754e+0 |  77.35    |  9.042    |  5.165    |  0.731    |
 Stopped after 20 iterations with train-auc = 0.940763 val-auc = 0.934383 ( diff = 0.006380 ) trai

In [0]:
AUCbest, ITERbest

(0.9355843333333332, 76)

In [0]:
xgb_bo.max['params']

{'colsample_bytree': 0.5246884658540595,
 'gamma': 0.9601535611794482,
 'learning_rate': 0.039066082460214306,
 'max_delta_step': 0.6124301463312987,
 'max_depth': 1.0955115772040507,
 'min_child_weight': 19.922110062205082,
 'n_estimators': 2323.540240695263,
 'num_leaves': 36.97092190520989,
 'reg_alpha': 0.48975141190259586,
 'reg_lambda': 1.5283443574008326,
 'subsample': 0.41995227297967597}

In [0]:
train_data = lgb.Dataset(data=train, label=target)
test_data = lgb.Dataset(test)

In [0]:
best_score = -1
def lgb_eval(n_estimators,learning_rate, num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight, colsample_bytree, min_child_samples,subsample):
    params = {
        'n_estimators': int(n_estimators), 
        'learning_rate':learning_rate,
        'metric': 'auc',
        'objective' : 'binary',
        'early_stopping_round':1000,
        "num_leaves" : int(round(num_leaves)),
        'feature_fraction': max(min(feature_fraction, 1), 0),
        'bagging_fraction': max(min(bagging_fraction, 1), 0),
        'max_depth': int(round(max_depth)),
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'min_split_gain': min_split_gain,
        'min_child_weight': min_child_weight,
        'colsample_bytree': colsample_bytree,
        'min_child_samples': int(min_child_samples),
        'subsample': subsample,
        'is_unbalance' :True
             }
    lgbm = lgb.cv(params, train_data, nfold=10, num_boost_round=1000, seed = 1993, stratified=True, verbose_eval =200)
      #     val_score = lgbm['test-auc-mean'].iloc[-1]
      #     train_score = lgbm['train-auc-mean'].iloc[-1]
    best_score = max(lgbm['auc-mean'])
    worst_score = min(lgbm['auc-mean'])
    print(' Stopped after %d iterations with best_train-auc = %f worst_train_auc = %f' % ( len(lgbm), best_score, worst_score ))
#     if ( val_score > AUCbest ):
#         AUCbest = val_score
#         ITERbest = len(xgbc)

    return best_score
    
    
    # range 
lgb_bo = BayesianOptimization(lgb_eval, {
                                        'n_estimators' :(1000, 3000),
                                        'learning_rate':(0.001, 0.5),
                                        'num_leaves': (20, 300),
                                        'feature_fraction': (0.4, 1.0),
                                        'bagging_fraction': (0.4, 1),
                                        'max_depth': (1, 15),
                                        'lambda_l1': (0, 10),
                                        'lambda_l2': (0, 10),
                                        'min_split_gain': (0.001, 0.1),
                                        'min_child_weight': (5, 50),
                                        'colsample_bytree' : (0.4,1.0),
                                        'min_child_samples' : (15,100),
                                        'subsample' : (0.4,1.0)
                                       }, random_state=1993)


In [0]:
import warnings
with warnings.catch_warnings():
     warnings.filterwarnings('ignore')
     lgb_bo.maximize(init_points=5, n_iter=25, acq='ei')

#lgb_bo.points_to_csv("bayes_opt_result.csv")

|   iter    |  target   | baggin... | colsam... | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_ch... | min_ch... | min_sp... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
[200]	cv_agg's auc: 0.935158 + 0.0138898
[400]	cv_agg's auc: 0.933715 + 0.0149412
[600]	cv_agg's auc: 0.931985 + 0.0153804
[800]	cv_agg's auc: 0.931431 + 0.0154033
[1000]	cv_agg's auc: 0.931431 + 0.0154033
[1200]	cv_agg's auc: 0.931431 + 0.0154033
 Stopped after 2 iterations with best_train-auc = 0.935291 worst_train_auc = 0.894815
|  1        |  0.9353   |  0.4603   |  0.5316   |  0.9586   |  3.314    |  2.045    |  0.08502  |  8.1      |  76.69    |  30.69    |  0.01928  |  2.918e+0 |  176.6    |  0.9033   |
[200]	cv_agg's auc: 0.893666 + 0.0192123
[400]	cv_agg's auc: 0.894237 + 0.0189766
[600]	cv_agg's auc: 0.894257 + 0.01891

AttributeError: ignored

In [0]:
lgb_bo.res[29]['params']

{'bagging_fraction': 0.4,
 'colsample_bytree': 0.4,
 'feature_fraction': 0.4,
 'lambda_l1': 0.0,
 'lambda_l2': 0.0,
 'learning_rate': 0.001,
 'max_depth': 15.0,
 'min_child_samples': 100.0,
 'min_child_weight': 50.0,
 'min_split_gain': 0.0010640577871197205,
 'n_estimators': 1519.699307429034,
 'num_leaves': 196.0237632077263,
 'subsample': 0.4}

In [0]:
cb_train = cb.Pool(data=train,
                  label=target)

In [0]:
best_score = -1
def cb_eval(bagging_temperature, max_depth, reg_lambda, colsample_bylevel):
    params = {
        'n_estimators': 150, 
        'learning_rate':0.05,
        'eval_metric' : 'AUC',
        'early_stopping_rounds':1000,
        'bagging_temperature': max(min(bagging_temperature, 1), 0),
        'max_depth': int(round(max_depth)),
        'reg_lambda': reg_lambda,
        'colsample_bylevel': colsample_bylevel
             }
    cv_rslt = cb.cv(cb_train, params, nfold=5, seed = 1993, stratified=True, verbose_eval =200)
    val_score = np.max(cv_rslt['test-AUC-mean'])
    #train_score = cv_rslt['train-AUC-mean'].iloc[-1]
    print(' Stopped after %d iterations with test-auc = %f ' % ( len(cv_rslt), val_score))

    return val_score
    
    
    # range 
cb_bo = BayesianOptimization(cb_eval, {
                                        'bagging_temperature': (0.2, 0.9),
                                        'max_depth': (5, 12),
                                        'reg_lambda': (0.1, 1),
                                        'colsample_bylevel': (0.3,0.9)
                                       }, random_state=1993)


In [0]:
import warnings
with warnings.catch_warnings():
     warnings.filterwarnings('ignore')
     cb_bo.maximize(init_points=5, n_iter=10, acq='ei')

#lgb_bo.points_to_csv("bayes_opt_result.csv")

|   iter    |  target   | baggin... | colsam... | max_depth | reg_la... |
-------------------------------------------------------------------------
0:	test: 0.8585937	best: 0.8585937 (0)	total: 5.06s	remaining: 12m 34s
149:	test: 0.9210755	best: 0.9232898 (83)	total: 9m 31s	remaining: 0us
 Stopped after 150 iterations with test-auc = 0.923290 
|  1        |  0.9233   |  0.2703   |  0.4316   |  11.52    |  0.3982   |
0:	test: 0.8487526	best: 0.8487526 (0)	total: 1.1s	remaining: 2m 44s
149:	test: 0.9214574	best: 0.9216033 (106)	total: 2m 14s	remaining: 0us
 Stopped after 150 iterations with test-auc = 0.921603 
|  2        |  0.9216   |  0.3432   |  0.401    |  8.55     |  0.7532   |
0:	test: 0.8268059	best: 0.8268059 (0)	total: 4.95s	remaining: 12m 17s
149:	test: 0.9194515	best: 0.9244863 (100)	total: 9m 49s	remaining: 0us
 Stopped after 150 iterations with test-auc = 0.924486 
|  3        |  0.9245   |  0.5996   |  0.4108   |  11.71    |  0.6033   |
0:	test: 0.8529176	best: 0.8529176 (

In [4]:
def rf_eval(n_estimators,learning_rate):
    params = {
          'n_estimators': int(n_estimators),
          'learning_rate' : learning_rate,
    }
        
#     y_pred_test_lgb=[]
#     y_pred_trainCV_lgb = np.zeros(train.shape[0])
#     y_pred_train_lgb = []
    err_rf = []
    from sklearn.model_selection import KFold,StratifiedKFold
    fold=StratifiedKFold(n_splits=10,shuffle=True,random_state=1993)
    i=1
    for train_index, test_index in fold.split(train,target):
        x_train, x_test = train.iloc[train_index], train.iloc[test_index]
        y_train, y_test = target[train_index], target[test_index]
        m = RandomForestClassifier(**params, random_state=1993)
        m.fit(x_train,y_train)
        preds=m.predict_proba(x_test)[:,-1]
        #print("err: ",roc_auc_score(y_test,preds))
        err_rf.append(roc_auc_score(y_test,preds))
        i=i+1
    print('AUC score: ',np.mean(err_rf,0))
    return np.mean(err_rf,0) 


rf_bo = BayesianOptimization(rf_eval,{
                                        'learning_rate':(0.01,10),
                                        'n_estimators':(100,1500)
                                      },
                             random_state=1993)
init_round=5
opt_round = 10
import warnings
with warnings.catch_warnings():
     warnings.filterwarnings('ignore')
     rf_bo.maximize(init_points=init_round, n_iter=opt_round)




# import seaborn as sns
# sns.set(rc={'figure.figsize':(11.7,8.27)})
# cat_feature_imp = pd.DataFrame([CatBoost.feature_names_ , CatBoost.feature_importances_]).T
# cat_feature_imp.columns = ["feature","varimp"]
# cat_feature_imp = cat_feature_imp.sort_values(by="varimp", ascending = False)
# sns.barplot(y="feature", x="varimp",data = cat_feature_imp, )
# plt.show()

|   iter    |  target   | learni... | n_esti... |
-------------------------------------------------


NameError: name 'train' is not defined

In [0]:
rf_bo.max['params']

{'learning_rate': 1.0132182062479367, 'n_estimators': 407.175900607226}